# Dynamic Programming.

It refers to simplifying a complicated problem by breaking it down into simpler sub-problem in a recursive manner. While some decision problems cannot be taken apart this way, decistons that span several points in time do often break apart recursively.

`Having optimal substructure`, a problem can be solved optimally by breaking it into sub-problems and then recursively finding the oprtimal solutions to the sub-problems.

If sub-problems can be nested recursively inside larger problems, so that dynamic programming methods are applicable, then there is a realation between the value of the larget problem and the values of the sub-problems.

**Dynamic programming helps us solve recursive problems with a highly-overlapping subproblem structure.** `Highly-overlapping` refers to the subproblems repeating again and again. In contrast, an alogrithm like mergesort recursively sorts `independent halves` of a list beforce combining the sorted halves. When the subproblems don't overlap, the algorithm is a divide-and-conquer alogrithm.

## Fibonacci numbers

As a reminder, the Fibonacci numbers are a sequence starting with 1, 1 where eath element in the sequence is the sume of the two previous elements: 1, 1, 2, 3, 5, 8, 13,....

In [2]:
def naive_fib(n):
    if n <= 1: return 1
    return naive_fib(n-1) + naive_fib(n-2)

In [3]:
naive_fib(10)

89

In [4]:
import timeit
timeit.timeit('naive_fib(20)', number=100, globals=globals())

0.8891706301365048

As we know, use the recursive way to calculate the Fibonacci numbers will occur overlapping calculation for certain subproblems. For example, when you want to know the value of `Fib(5)`, you need to calculate `Fib(4)` and `Fib(3)` first, but the `Fib(4)` will come from the value of `Fib(3)`. The fact is we need to produce the result of `Fib(3)` twice. The total runtime will be exponential in n as `O(2^n)`. As mentioned above, the recursive subproblem of produce Fibonacci number have an overlapping calculation, we can use the DP to improve our runtime.

**Using memoization**

In [5]:
def memo_fib(n, memo = {}):
    if n <= 1:
        # Store the fib(0) and fib(1) into memo.
        memo[n] = 1
    if n in memo:
        # If fib(n) in the memo, do not reproduce the same fib number.
        return memo[n]
    fib_n = memo_fib(n - 1, memo) + memo_fib(n - 2, memo)
    memo[n] = fib_n
    return fib_n

In [6]:
memo_fib(10)

89

In [7]:
timeit.timeit('memo_fib(100)', number=100, globals=globals())

0.00018539209850132465

As the time showing above, we have significantly reduced our runtime bound from `O(2^n)` to `O(n)`. Otherwise, the space-time will be `O(n)` because we store the result of the previous value of the Fibonacci number.

**Bottom-up approach**

The formula of the Fibonacci number can be write as `Fib(n) = Fib(n-1) + Fib(n+2)`. If we consider that a `Fib(i)` is the subproblem of `Fib(n)`, where `i <= n`, we can only need the result of `Fib(i-1)` and `Fib(i-2)` to produce its result. That means we only need two variables to store our values, and we can throw away the value before the `Fib(i-2)` as its no longer help us to calculate the `Fib(i)`.

In [8]:
def bottom_up_fib(n):
    if n <= 1:
        return 1
    fib1 = 1
    fib2 = 1
    for i in range(2, n+1):
        # the old "fib1" will be throwd and assign the old "fib2" to it.
        tmp = fib1
        fib1 = fib2
        fib2 = tmp + fib2
    return fib2

In [9]:
bottom_up_fib(10)

89

In [10]:
timeit.timeit('bottom_up_fib(100)', number=100, globals=globals())

0.0026252679526805878

# The House Robber Problem

In the **House Robber Problem**, you are a robber who has found a block of house to rob. Each house i has a non-negative v(i) worth of value inside that you can steal. However, due to the security systems of the houses are connected, you'll get caught if you rob two adjacent houses. What's the maximum value you can steal from the block?

**Example 1:**
```py
Input: [1,2,3,1]
Output: 4
Explanation: Rob house 1 (money = 1) and then rob house 3 (money = 3).
             Total amount you can rob = 1 + 3 = 4.
```
**Example 2:**
```py
Input: [2,7,9,3,1]
Output: 12
Explanation: Rob house 1 (money = 2), rob house 3 (money = 9) and rob house 5 (money = 1).
             Total amount you can rob = 2 + 9 + 1 = 12.
```
Let's use the same steps solve this problem like Fibonacci.

1. The first step in solveing a dynamic programming problem is **identifying the subproblems**. Whenever we encounter a house i, we have two choices:
  * Steal from the house i add the `v[i]` into your wealth, and then compare it with the value of stealing the house up to `i - 2`, because you can not steal the adjacent houses. The formula can be write like `rob(i) = rob(i-2) + v[i]`.
  * Don't steal the house i, in which case you're free to maximum the stolen value up to house `i-1`. And you add nothing to you wealth. The formula can be write like `rob(i) = rob(i - 1)`.

2. The second step, think about our base case. When the houses are empty, the stolen value will be 0, and is the house is only one, we will return its value, because the better choice is to steal it.

3. The third step, Define a recurrence relation.

Define `rob(i)` to be the result of our problem which is maximum value that can be stolen if only stealing from the houses 0 to i.

**Then our relation formula will be like: `rob(i) = max(rob(i-1), rob(i-2) + v[i]) and (rob(0) = 0, rob(1) = v[1])`.** And the `v[i]` is the value of the house i.

In [11]:
def naive_rob(houses):
    if not houses:
        return 0
    if len(houses) == 1:
        return houses[0]
    return max(naive_rob(houses[2:]) + houses[0], naive_rob(houses[1:]))

In [12]:
timeit.timeit('naive_rob([2,7,9,3,1])', number=100, globals=globals())

0.001957674976438284

**Use the memorization**

From the relation formula, we know that there will exist the overlapping calculation from `rob(i)`. As `rob(i)` need results of `rob(i-1)` and `rob(i-2)`, and `rob(i-1)`  also need result of `rob(i-2)`. So use the memorization skill to improve our solution will quickly come to our mind. 

In [13]:
def memo_rob(houses, memo={}):
    if not houses:
        return 0
    if len(houses) == 1:
        return houses[0]
    if len(houses) in memo:
        # If subproblem in the memo, do not calculate the same subproblem.
        return memo[len(houses)]
    rob_i = max(memo_rob(houses[2:], memo) + houses[0], memo_rob(houses[1:], memo))
    memo[len(houses)] = rob_i
    return rob_i

In [14]:
timeit.timeit('memo_rob([2,7,9,3,1])', number=1000, globals=globals())

0.001558423973619938

**Use bottom-up approach**

As the relation formual shows ue `rob(i) = max(rob(i-1), rob(i-2) + v[i]) and (rob(0) = 0, rob(1) = v[1])`, we can notice that, the intermidate produced value can be throwed as its no need to be used on the current calculation.

In [15]:
def bottom_up_rob(houses):
    if not houses:
        return 0
    if len(houses) == 1:
        return houses[0]
    rob_0 = 0
    rob_1 = houses[0]
    for i in range(1, len(houses)):
        tmp = rob_1
        rob_1 = max(rob_0 + houses[i], rob_1)
        rob_0 = tmp
    return rob_1

In [16]:
timeit.timeit('bottom_up_rob([2,7,9,3,1])', number=1000, globals=globals())

0.006975776981562376

Both of Fibonacci and Robber problem is `one-dimensinal` problems, in which we just need to iterate through a linear sequence of subproblems.
# Coin Change

You are given coins of different denominations and a total amount of money amount. Write a function to compute the fewest number of coins that you need to make up that amount. If that amount of money cannot be made up by any combination of the coins, return -1.

**Example 1:**
```py
Input: coins = [1, 2, 5], amount = 11
Output: 3 
Explanation: 11 = 5 + 5 + 1
```
**Example 2:**
```py
Input: coins = [2], amount = 3
Output: -1
```

1. First, define the subproblem, at any given point in the algorithm, we will consider a subset of the denominations `coins`, and the amount we want to add up to. Let's bring us two choice to make.
  * We not to use the first coin on our coins set. And we use the same target amount to find the lowest number of coins to make the change. Our coins set will extract the first one out. The relation formula will be write as `change(coins, amount) = change(coins[1:], amount)`.
  * Use the first coin and decreases the target amount correspond on the denomination of the first coin. And do not change our coins set, which give us a chance to use the same coin on the next recursion. If we choose this coin to decrease our amount we need to increase the count of change coin by one. The relation formula will be write as `change(coins, amount) = change(coins, amount - coins[0])`.
  
2. Second, figure out our base case where we need to stop our recursive. 
  * When the coins set is one, which means we can only use one coin to change our amount, so if the reminder of amount divide the denomination of coin is zero, the divide value is our count, otherwise, we will return MAX, there is no way to change the amount with this coin.
  * When the amount is zero, there are no need to change our amount anymore, so our count return zero.
  * As we don't know whether our coins set if sorted or not, we need filter the coins set by comparing the first coin denomination with our target amount or not. If it's greater than our target amount, we can eliminate it from our coins set.

3. Third, the recursion relation formula of this problem can be write as `change(coins, amount) = min(change(coins[1:], amount), change(coins, amount - coins[0]))`.

In [17]:
def naive_make_change(coins, amount):
    
    if len(coins) == 1:
        if amount % coins[0]:
            return float('inf')
        count = amount // coins[0]
        return count
    
    if amount == 0:
        return 0
    
    if amount < coins[0]:
        return naive_make_change(coins[1:], amount)
    
    return min(naive_make_change(coins[1:], amount), naive_make_change(coins, amount-coins[0]) + 1)

In [18]:
naive_make_change([1, 2, 5], 11)

3

In [19]:
timeit.timeit('naive_make_change([1, 2, 5], 11)', number=1000, globals=globals())

0.13363851001486182

**Use the memorization**

We use the map to store our recursion result, as its `O(1)` runtime to search our previour calculate value. And we use the current state of our recursion function arguments as the key to store correspond counts.

In [20]:
def memo_make_change(coins, amount, memo={}):
    
    if len(coins) == 1:
        if amount % coins[0]:
            return float('inf')
        count = amount // coins[0]
        return count
    
    if amount == 0:
        return 0
    
    if (coins[0], amount) in memo:
        # If subproblem in the memo, do not calculate the same subproblem.
        return memo[(coins[0], amount)]
    
    if amount < coins[0]:
        return memo_make_change(coins[1:], amount, memo)
    
    change_n = min(memo_make_change(coins[1:], amount, memo), memo_make_change(coins, amount-coins[0], memo) + 1)
    memo[(coins[0], amount)] = change_n
    return change_n

In [21]:
memo_make_change([1, 2, 5], 11)

3

In [22]:
timeit.timeit('memo_make_change([370,417,408,156,143,434,168,83,177,280,117], 9953)', number=1000, globals=globals())

0.3491729542147368

**Bottom up approah**

`F(S)` - minimum number of coins needed to make change for amount S using coin denominations `coins[c0, c1, ... ,cn-1]`.
Our recursion relation can be redefine like, `F(S) = min F(S - ci) + 1 for ci in range coins and S - ci >= 0`. Also our base case can be like `F(S) == 0, when S == 0`. We consider our problem from bottom, when our amount = 1, and our coins set is [1, 2, 3]. the result of `F(1) = F(1 - 1) + 1` will be `1`, and when amount is 2, we will have `F(2) = min (F(2-1), F(2-2))` will is 1.
Ans we can fill out our two dimension table of as following.

|amount\coins |1 |2 |3| F(i)|
|------|:------|:--------|:--------|:---|
|1|F(0)|0|0|1|
|2|F(1)|F(0)|0|1|
|3|F(2)|F(1)|F(0)|1|
|4|F(3)|F(2)|F(1)|2|
|5|F(4)|F(3)|F(2)|2|
|6|F(5)|F(4)|F(3)|2|

In [23]:
def bottom_up_make_change(coins, amount):
    max_count = amount + 1
    dp = [max_count for _ in range(amount+1)]
    dp[0] = 0
    for i in range(1, amount+1):
        for j in range(len(coins)):
            if coins[j] <= i:
                dp[i] = min(dp[i], dp[i - coins[j]] + 1)
    return dp[amount]

In [24]:
bottom_up_make_change([1, 2, 5], 11)

3

In [25]:
timeit.timeit('bottom_up_make_change([370,417,408,156,143,434,168,83,177,280,117], 9953)', number=10, globals=globals())

1.6425782199949026

## Unique Paths

A robot is located at the top-left corner of a m x n grid (marked 'Start' in the diagram below).

The robot can only move either down or right at any point in time. The robot is trying to reach the bottom-right corner of the grid (marked 'Finish' in the diagram below).

How many possible unique paths are there?


![alt_text](robot_maze.png )

Above is a 7 x 3 grid. How many possible unique paths are there?


* Note: m and n will be at most 100.

Example 1:
```
Input: m = 3, n = 2
Output: 3
```
Explanation:
From the top-left corner, there are a total of 3 ways to reach the bottom-right corner:
1. Right -> Right -> Down
2. Right -> Down -> Right
3. Down -> Right -> Right


Example 2:
```
Input: m = 7, n = 3
Output: 28
```

Use recursion, the robot each step has two chioces, went down and went right. We can go deep let robit find the subproble of m-1, n or m, n-1.

In [39]:
import pysnooper
def uniquePaths(m, n):
    count = 0
    def recursion(m, n):
        nonlocal count
        if m == 1 and n == 1:
            count += 1
            return
        if m != 1:
            recursion(m-1, n)
        if n != 1:
            recursion(m, n-1)
    recursion(m, n)
    return count

In [40]:
uniquePaths(7, 3)

28

In [44]:
def uniquePathsQ(m, n):
    queue = [(m, n)]
    ans = 0
    while queue:
        node = queue.pop()
        if node[0] == 1 and node[1] == 1:
            ans += 1
            continue
        if node[0]:
            queue.append((node[0]-1, node[1]))
        if node[1]:
            queue.append((node[0], node[1]-1))
    return ans

In [64]:
def uniquePathDy(m, n):
    cache = [[1] * n for i in range(m)]
    for i in range(m):
        if i > 0:
            for j in range(n):
                if j > 0:
                    cache[i][j] = cache[i-1][j] + cache[i][j-1]
    return cache[-1][-1]

In [65]:
uniquePathDy(3, 7)

28

In [69]:
def uniquePathDyOneRow(m, n):
    cache = [1] * n
    for i in range(m):
        if i > 0:
            for j in range(n):
                if j > 0:
                    cache[j] = cache[j-1] + cache[j]
    return cache[-1]

In [72]:
uniquePathDyOneRow(2, 3)

3

## Unique Paths II

A robot is located at the top-left corner of a m x n grid (marked 'Start' in the diagram below).

The robot can only move either down or right at any point in time. The robot is trying to reach the bottom-right corner of the grid (marked 'Finish' in the diagram below).

Now consider if some obstacles are added to the grids. How many unique paths would there be?

An obstacle and empty space is marked as 1 and 0 respectively in the grid.

Note: m and n will be at most 100.

Example 1:
```
Input:
[
  [0,0,0],
  [0,1,0],
  [0,0,0]
]
Output: 2
```

Explanation:
There is one obstacle in the middle of the 3x3 grid above.
There are two ways to reach the bottom-right corner:

1. Right -> Right -> Down -> Down
2. Down -> Down -> Right -> Right

In [96]:
def uniquePathsWithObstacles(obstacleGrid):
    path = [0] * len(obstacleGrid[0])
    for i, v in enumerate(obstacleGrid):
        if i == 0:
            for m in range(len(v)):
                if v[m] == 1:
                    break
                else:
                    path[m] = 1
        else:
            for j, n in enumerate(v):
                if j == 0:
                    path[j] = 0 if n == 1 else path[j]
                elif n == 1:
                    path[j] = 0
                else:
                    path[j] = path[j] + path[j-1]
    return path[-1]

In [97]:
uniquePathsWithObstacles([
  [0,1,0],
  [0,0,0],
  [0,0,0]
])

3

Input: workers = [[0,0],[1,1],[2,0]], bikes = [[1,0],[2,2],[2,1]]
Output: 6
Explanation: 
We assign bike 0 to worker 0, bike 1 to worker 1. The Manhattan distance of both assignments is 3, so the output is 6.

In [68]:
import pysnooper
@pysnooper.snoop()
def assignBikes(workers, bikes):
    result = float("inf")
    leave_bikes = len(bikes) - len(workers) 
    def backtrack(workers , bikes, ans=0):
        nonlocal result
        if not workers and len(bikes) == leave_bikes:
            result = min(ans, result)
        for i, w in enumerate(workers):
            for j, b in enumerate(bikes):
                ans += abs(w[0] - b[0]) + abs(w[1] - b[1])
                backtrack(workers[i+1:], bikes[:j] + bikes[j+1:], ans)
                ans -= abs(w[0] - b[0]) + abs(w[1] - b[1])
    backtrack(workers, bikes)
    return result

In [70]:
assignBikes([[0,0],[1,0],[2,0],[3,0],[4,0],[5,0],[6,0]],
[[0,999],[1,999],[2,999],[3,999],[4,999],[5,999],[6,999],[7,999],[8,999],[9,999]])

Starting var:.. workers = [[0, 0], [1, 0], [2, 0], [3, 0], [4, 0], [5, 0], [6, 0]]
Starting var:.. bikes = [[0, 999], [1, 999], [2, 999], [3, 999], [4, 999], [5, 999], [6, 999], [7, 999], [8, 999], [9, 999]]
10:45:30.080743 call         3 def assignBikes(workers, bikes):
10:45:30.080894 line         4     result = float("inf")
New var:....... result = inf
10:45:30.081018 line         5     leave_bikes = len(bikes) - len(workers) 
New var:....... leave_bikes = 3
10:45:30.081129 line         6     def backtrack(workers , bikes, ans=0):
New var:....... backtrack = <function assignBikes.<locals>.backtrack at 0x7f55fe67b0d0>
10:45:30.081244 line        15     backtrack(workers, bikes)
Modified var:.. result = 6993
10:45:55.690099 line        16     return result
10:45:55.690238 return      16     return result
Return value:.. 6993


6993

In [34]:
import pysnooper
@pysnooper.snoop()
def assignBikes(workers, bikes):
    store = {}
    distance = lambda w, b: abs(w[0] - b[0]) + abs(w[1] - b[1])
    def dfs(worker_no, bikes_array):
        nonlocal bikes, distance, workers
        min_dis = float('inf')
        if worker_no == len(workers):
            return 0
        if (worker_no, tuple(bikes_array)) in store:
            return store[(worker_no, tuple(bikes_array))]
        for i, b in enumerate(bikes_array):
            if b == 0:
                bikes_array[i] = 1
                min_dis = min(
                    min_dis, distance(workers[worker_no], bikes[i]) + dfs(worker_no+1, bikes_array))
                bikes_array[i] = 0
        store[(worker_no, tuple(bikes_array))] = min_dis
        return min_dis
    return dfs(0, [0] * len(bikes))

In [36]:
assignBikes([[0,0],[1,0],[2,0],[3,0],[4,0],[5,0],[6,0]],
[[0,999],[1,999],[2,999],[3,999],[4,999],[5,999],[6,999],[7,999],[8,999],[9,999]])

Starting var:.. workers = [[0, 0], [1, 0], [2, 0], [3, 0], [4, 0], [5, 0], [6, 0]]
Starting var:.. bikes = [[0, 999], [1, 999], [2, 999], [3, 999], [4, 999], [5, 999], [6, 999], [7, 999], [8, 999], [9, 999]]
14:30:37.029766 call         3 def assignBikes(workers, bikes):
14:30:37.029882 line         4     store = {}
New var:....... store = {}
14:30:37.029998 line         5     distance = lambda w, b: abs(w[0] - b[0]) + abs(w[1] - b[1])
New var:....... distance = <function assignBikes.<locals>.<lambda> at 0x7f07fab632f0>
14:30:37.030116 line         6     def dfs(worker_no, bikes_array):
New var:....... dfs = <function assignBikes.<locals>.dfs at 0x7f07fab63e18>
14:30:37.030265 line        21     return dfs(0, [0] * len(bikes))
Modified var:.. store = {(6, (1, 1, 1, 1, 1, 1, 0, 0, 0, 0)): 999, (6, (... 5994, (0, (0, 0, 0, 0, 0, 0, 0, 0, 0, 0)): 6993}
14:30:37.138521 return      21     return dfs(0, [0] * len(bikes))
Return value:.. 6993


6993

In [40]:
nums = [10, 2, 3, 4]
import itertools
for i in itertools.combinations(nums, 2):
    print(i)

(10, 2)
(10, 3)
(10, 4)
(2, 3)
(2, 4)
(3, 4)


In [99]:
def numPairsDivisibleBy60(time):
    store = [0] * 60
    for i, t in enumerate(time):
        store[t%60] += 1
    ans = 0
    for j in range(31):
        if j == 0:
            ans += store[j] * (store[j] - 1) // 2
        elif j == 30:
            ans += store[j] * (store[j] - 1) // 2
        elif store[j] and store[60 - j]:
            ans += store[j] * store[60 - j]
    return ans

In [100]:
numPairsDivisibleBy60([174,188,377,437,54,498,455,239,183,347,59,199,52,488,147,82])

8 2 1


2

In [128]:
for i in range('world', 'woe'):
    print(i)

world
woe


In [140]:
def isAlienSorted(words, order):
    order = {o: i for i, o in enumerate(order)}
    for i in range(len(words) - 1):
        w1 = words[i]
        w2 = words[i+1]
        for v in range(min(len(w1), len(w2))):
            if w1[v] != w2[v]:
                if order[w1[v]] > order[w2[v]]:
                    return False
                break
        else:
            if len(w1) > len(w2):
                return False
    return True